# Library / Packages

In [1]:
# basic
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde

# data preparation
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 

# data modeling
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# data scoring
from sklearn.metrics import classification_report

# data tuning   
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split

# visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# Format

In [2]:
def lab_round(x, pos): 
    if abs(x) >= 1e9: 
        return f'{x/1e9} B'
    
    elif abs(x) >= 1e6:
        return f'{x/1e6} M'
    
    elif abs(x) >= 1e3:
        return f'{x/1e3} K'
    
    else:
        return f'{x}'
    
def val_round(x):
    if abs(x) >= 1e9:
        return f'{x/1e9:.2f} B'
    
    elif abs(x) >= 1e6:
        return f'{x/1e6:.2f} M'
    
    elif abs(x) >= 1e3:
        return f'{x/1e3:.2f} K'
    
    else:
        return f'{x:.2f}'

In [3]:
def filter_outliers_iqr(df, columns = None, threshold = 1.5):
    # Jika tidak ada kolom yang ditentukan, gunakan semua kolom numerik
    if columns is None:
        columns = df.select_dtypes(include = ["number"]).columns.tolist()
    
    # Salin DataFrame untuk memastikan tidak ada modifikasi langsung
    df_filtered = df.copy()
    
    for column in columns:
        # Hitung Q1, Q3, dan IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Hitung batas bawah dan atas
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        # Hapus baris dengan outlier
        df_filtered = df_filtered[(df_filtered[column] >= lower_bound) & (df_filtered[column] <= upper_bound)]
    
    return df_filtered

In [4]:
# Fungsi untuk konversi tipe data
def convert_object_columns_to_numeric(df):
    for col in df.select_dtypes(include = ['object']).columns:  
        try:
            # Cek apakah semua nilai bisa dikonversi ke float
            df[col] = pd.to_numeric(df[col], errors='raise')
            
            # Jika bisa, ubah ke int jika semua nilai adalah bilangan bulat
            if all(df[col] % 1 == 0):  # Cek apakah semua nilai adalah bilangan bulat
                df[col] = df[col].astype(int)

        except ValueError:
            pass  # Jika ada nilai non-angka, biarkan tetap object
        
    return df

# Dataset

In [5]:
# Memuat data train dan test
train_df = pd.read_csv('../dataset/train.csv')
test_df = pd.read_csv('../dataset/test.csv')

In [6]:
# show all column
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Train Dataset

In [ ]:
train_df.info()

In [ ]:
# drop column
train_df = train_df.drop('Id', axis = 1)

# convert object if all numeric
train_df = convert_object_columns_to_numeric(train_df)

# check duplicate general data
print(f'Total General Duplicated: {train_df.duplicated().sum()} \n')
train_df.info()

In [ ]:
train_df.head()

In [ ]:
# null column
null_numeric = []
null_obj = []

# 
null_columns = train_df.columns[train_df.isnull().sum() > 0]

for col in null_columns:
    if train_df[col].dtype in ['int', 'float']:
        null_numeric.append(col)
        
    elif train_df[col].dtype == 'object':
        null_obj.append(col)

# 
print("Null Numeric:", null_numeric)
print("Null String:", null_obj)

In [ ]:
# 
num_cols = []
obj_cols = []

for col in train_df:
    if train_df[col].dtype in ['int', 'float']:
        num_cols.append(col)
        
    elif train_df[col].dtype == 'object':
        obj_cols.append(col)

# 
print("Numeric Cols:", num_cols)
print("String Cols:", obj_cols)

In [12]:
# Original columns
train_original = train_df.columns

# Numeric Pipeline
numerical_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "mean"))
])

# String Pipeline
categorical_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "most_frequent"))
])

# ColumnTransformer untuk menggabungkan proses imputasi
prep_stage_1 = ColumnTransformer(
    transformers = [
        ("num", numerical_pipeline, num_cols), 
        ("cat", categorical_pipeline, obj_cols), 
    ], 
    remainder = "drop", 
    verbose_feature_names_out = True)

In [13]:
# Transform data menggunakan fit_transform pada tahap 1
train_df = prep_stage_1.fit_transform(train_df)

# Columns After: ubah kembali ke DataFrame dengan kolom dari prep_stage_1
train_df = pd.DataFrame(train_df, columns = prep_stage_1.get_feature_names_out())

# Hilangkan prefix (misalnya, "num__", "cat__", "out__")
clean_columns = [col.split("__", 1)[-1] for col in train_df.columns]
train_df.columns = clean_columns

In [ ]:
# Menampilkan total null pada setiap kolom
null_columns = train_df.isnull().sum()[train_df.isnull().sum() > 0]
print(f'Total null columns: {null_columns} \n')
train_df.info()

In [ ]:
# change object after transform
train_df = convert_object_columns_to_numeric(train_df)
train_df.info()

In [ ]:
# Cetak jumlah baris sebelum filter
print(f"Total Rows Before Filtering: {len(train_df)}")

# Pilih kolom numerik
num_cols = train_df.select_dtypes(include = ["number"]).columns

# Terapkan filter pada kolom numerik
train_df = filter_outliers_iqr(train_df, columns = num_cols)

# Cetak jumlah baris setelah filter
print(f"Total Rows After Filtering: {len(train_df)}")

In [ ]:
# Daftar kolom untuk label encoding (kolom ordinal)
encoding_set = {'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
                'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 
                'FireplaceQu', 'GarageQual', 'GarageCond'}

# Inisialisasi list untuk menyimpan kolom yang telah dikelompokkan
train_ordinal_cols = []
train_one_hot_cols = []
train_numeric_cols = []

# Mengelompokkan kolom berdasarkan tipe data
for col in train_df.columns:
    if train_df[col].dtype in ['int', 'float']:
        train_numeric_cols.append(col)

    elif train_df[col].dtype == 'object':
        if col in encoding_set:
            train_ordinal_cols.append(col)

        else:
            train_one_hot_cols.append(col)

# Menampilkan hasil
print("Ordinal Encoding Columns:", train_ordinal_cols)
print("One-Hot Encoding Columns:", train_one_hot_cols)
print("Numeric Columns:", train_numeric_cols)

## Test Dataset

In [ ]:
test_df.info()

In [ ]:
# drop column
test_df = test_df.drop('Id', axis = 1)

# convert object if all numeric
test_df = convert_object_columns_to_numeric(test_df)

# check duplicate general data
print(f'Total General Duplicated: {test_df.duplicated().sum()} \n')
test_df.info()

In [ ]:
# null column
null_numeric = []
null_obj = []

# 
null_columns = test_df.columns[test_df.isnull().sum() > 0]

for col in null_columns:
    if test_df[col].dtype in ['int', 'float']:
        null_numeric.append(col)
        
    elif test_df[col].dtype == 'object':
        null_obj.append(col)

# 
print("Null Numeric:", null_numeric)
print("Null String:", null_obj)

In [ ]:
# 
num_cols = []
obj_cols = []

for col in test_df:
    if test_df[col].dtype in ['int', 'float']:
        num_cols.append(col)
        
    elif test_df[col].dtype == 'object':
        obj_cols.append(col)

# 
print("Numeric Cols:", num_cols)
print("String Cols:", obj_cols)

In [22]:
# Original columns
test_original = test_df.columns

# Numeric Pipeline
numerical_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "mean"))
])

# String Pipeline
categorical_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "most_frequent"))
])

# ColumnTransformer untuk menggabungkan proses imputasi
prep_stage_1 = ColumnTransformer(
    transformers = [
        ("num", numerical_pipeline, num_cols), 
        ("cat", categorical_pipeline, obj_cols), 
    ], 
    remainder = "drop", 
    verbose_feature_names_out = True)

In [23]:
# Transform data menggunakan fit_transform pada tahap 1
test_df = prep_stage_1.fit_transform(test_df)

# Columns After: ubah kembali ke DataFrame dengan kolom dari prep_stage_1
test_df = pd.DataFrame(test_df, columns = prep_stage_1.get_feature_names_out())

# Hilangkan prefix (misalnya, "num__", "cat__", "out__")
clean_columns = [col.split("__", 1)[-1] for col in test_df.columns]
test_df.columns = clean_columns

In [ ]:
# Menampilkan total null pada setiap kolom
null_columns = test_df.isnull().sum()[test_df.isnull().sum() > 0]
print(f'Total null columns: {null_columns} \n')
test_df.info()

In [ ]:
# change object after transform
test_df = convert_object_columns_to_numeric(test_df)
test_df.info()

In [ ]:
# Cetak jumlah baris sebelum filter
print(f"Total Rows Before Filtering: {len(test_df)}")

# Pilih kolom numerik
num_cols = test_df.select_dtypes(include = ["number"]).columns

# Terapkan filter pada kolom numerik
test_df = filter_outliers_iqr(test_df, columns = num_cols)

# Cetak jumlah baris setelah filter
print(f"Total Rows After Filtering: {len(test_df)}")

In [ ]:
# Daftar kolom untuk label encoding (kolom ordinal)
encoding_set = {'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
                'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 
                'FireplaceQu', 'GarageQual', 'GarageCond'}

# Inisialisasi list untuk menyimpan kolom yang telah dikelompokkan
test_ordinal_cols = []
test_one_hot_cols = []
test_numeric_cols = []

# Mengelompokkan kolom berdasarkan tipe data
for col in test_df.columns:
    if test_df[col].dtype in ['int', 'float']:
        test_numeric_cols.append(col)

    elif test_df[col].dtype == 'object':
        if col in encoding_set:
            test_ordinal_cols.append(col)

        else:
            test_one_hot_cols.append(col)

# Menampilkan hasil
print("Ordinal Encoding Columns:", test_ordinal_cols)
print("One-Hot Encoding Columns:", test_one_hot_cols)
print("Numeric Columns:", test_numeric_cols)

## Split Set

In [ ]:
# Identifikasi kolom-kolom yang ada di train dan test
ordinal_encoding_cols = list(set(train_ordinal_cols) & set(test_ordinal_cols))
one_hot_encoding_cols = list(set(train_one_hot_cols) & set(test_one_hot_cols))
numeric_cols = list(set(train_numeric_cols) & set(test_numeric_cols))

# 
print(f'ordinal cols: {ordinal_encoding_cols}')
print(f'one-hot cols: {one_hot_encoding_cols}')
print(f'numeric cols: {numeric_cols}')

In [29]:
# transform
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)
ordinal_transformer = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

prep_stage_2 = ColumnTransformer(
    transformers = [
        ("num", numerical_transformer, numeric_cols), 
        ("cat", categorical_transformer, one_hot_encoding_cols), 
        ("ord", ordinal_transformer, ordinal_encoding_cols)
    ], remainder = "passthrough")

### Train Data

In [ ]:
# Transform data
transformed_data = prep_stage_2.fit_transform(train_df)

# Membuat DataFrame dengan kolom hasil transformasi
# ====================================================
# Mendapatkan nama kolom baru untuk OneHotEncoder
categorical_feature_names = prep_stage_2.named_transformers_["cat"].get_feature_names_out(one_hot_encoding_cols)

# Gabungkan semua nama kolom
all_columns = (
    numeric_cols +
    list(categorical_feature_names) +
    ordinal_encoding_cols +
    list(train_df.columns.difference(numeric_cols + one_hot_encoding_cols + ordinal_encoding_cols))
)

# Membuat DataFrame dengan nama kolom yang sesuai
train_df = pd.DataFrame(transformed_data, columns = all_columns)

# Menampilkan total null pada setiap kolom
null_columns = train_df.isnull().sum()[train_df.isnull().sum() > 0]
print(f'Train Stage 2 Check: {null_columns}')
train_df.head(3)

In [ ]:
train_df.info()

### Test Data

In [ ]:
# Transform data
transformed_data = prep_stage_2.fit_transform(test_df)

# Membuat DataFrame dengan kolom hasil transformasi
# ====================================================
# Mendapatkan nama kolom baru untuk OneHotEncoder
categorical_feature_names = prep_stage_2.named_transformers_["cat"].get_feature_names_out(one_hot_encoding_cols)

# Gabungkan semua nama kolom
all_columns = (
    numeric_cols +
    list(categorical_feature_names) +
    ordinal_encoding_cols +
    list(test_df.columns.difference(numeric_cols + one_hot_encoding_cols + ordinal_encoding_cols))
)

# Membuat DataFrame dengan nama kolom yang sesuai
test_df = pd.DataFrame(transformed_data, columns = all_columns)

# Menampilkan total null pada setiap kolom
null_columns = test_df.isnull().sum()[test_df.isnull().sum() > 0]
print(f'Test Stage 2 Check: {null_columns}')
test_df.tail(3)

In [ ]:
test_df.info()

### Data for modeling

In [ ]:
# Memisahkan kolom target dari data
# Mencari kolom target di train_df
train_target_col = train_df.filter(like='MSZoning_').columns

if len(train_target_col) > 0:
    train_target_col = train_target_col[0]  # Mengambil kolom pertama yang cocok
    
    # Pastikan kolom target ada di train_df
    if train_target_col in train_df.columns:
        X_train = train_df.drop(columns=[train_target_col])
        y_train = train_df[train_target_col]
    else:
        raise ValueError(f"Kolom target '{train_target_col}' tidak ditemukan di train_df.")
else:
    raise ValueError("Kolom dengan filter 'MSZoning_' tidak ditemukan di train_df.")

# Mencari kolom target di test_df
test_target_col = test_df.filter(like='MSZoning_').columns

if len(test_target_col) > 0:
    test_target_col = test_target_col[0]  # Mengambil kolom pertama yang cocok
    
    # Pastikan kolom target ada di test_df
    if test_target_col in test_df.columns:
        X_test = test_df.drop(columns=[test_target_col])
        y_test = test_df[test_target_col]
    else:
        raise ValueError(f"Kolom target '{test_target_col}' tidak ditemukan di test_df.")
else:
    raise ValueError("Kolom dengan filter 'MSZoning_' tidak ditemukan di test_df.")

# Validasi tambahan
print("Shape X_train:", X_train.shape)
print("Shape y_train:", y_train.shape)
print("Shape X_test:", X_test.shape)
print("Shape y_test:", y_test.shape)

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
X_test.head(3)

In [ ]:
y_test.head(3)

In [39]:
# # Memisahkan kolom target dari data
# target_col = train_df.filter(like = 'MSZoning_').columns

# # Memastikan kolom target ada di dalam DataFrame sebelum mencoba memisahkannya
# if target_col in train_df.columns:
#     X_train = train_df.drop(columns = [target_col])
#     y_train = train_df[target_col]

# else:
#     X_train = train_df
#     y_train = None

# if target_col in test_df.columns:
#     X_test = test_df.drop(columns = [target_col])
    
# else:
#     X_test = test_df

# Modeling

## Set Parameter

In [40]:
# Membuat pipeline model
pipelines = {
    'RandomForest': Pipeline(steps = [
        ('pca', PCA()),
        ('classifier', RandomForestClassifier(random_state = 42))
    ]),

    'LogisticRegression': Pipeline(steps = [
        ('pca', PCA()),
        ('classifier', LogisticRegression(max_iter = 1000, random_state = 42))
    ]),
    
    'SVC': Pipeline(steps = [
        ('pca', PCA()),
        ('classifier', SVC(random_state = 42))
    ]),
    
    'GradientBoosting': Pipeline(steps = [
        ('pca', PCA()),
        ('classifier', GradientBoostingClassifier(random_state = 42))
    ]),
    
    'XGBoost': Pipeline(steps = [
        ('pca', PCA()),
        ('classifier', XGBClassifier(use_label_encoder = False, eval_metric = 'mlogloss', random_state = 42))
    ])
}

In [41]:
# Parameter grid untuk GridSearchCV
param_grids = {
    'RandomForest': {
        'pca__n_components': [0.90, 0.95, 0.99],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__bootstrap': [True, False],
    },
    'LogisticRegression': {
        'pca__n_components': [0.90, 0.95, 0.99],
        'classifier__penalty': ['l1', 'l2', 'elasticnet'],  # Sesuaikan penalti yang valid
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__solver': ['liblinear', 'saga'],  # Solver yang mendukung kombinasi penalti
        'classifier__max_iter': [100, 500, 1000],
        'classifier__l1_ratio': [0.1, 0.5, 0.9],  # Hanya relevan untuk penalti 'elasticnet'
    },
    'SVC': {
        'pca__n_components': [0.90, 0.95, 0.99],
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'classifier__degree': [2, 3, 4],
        'classifier__gamma': ['scale', 'auto'],
    },
    'GradientBoosting': {
        'pca__n_components': [0.90, 0.95, 0.99],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__max_depth': [3, 5, 7],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__subsample': [0.8, 0.9, 1.0],
    },
    'XGBoost': {
        'pca__n_components': [0.90, 0.95, 0.99],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__max_depth': [3, 5, 7],
        'classifier__min_child_weight': [1, 3, 5],
        'classifier__gamma': [0, 0.1, 0.5],
        'classifier__subsample': [0.8, 0.9, 1.0],
        'classifier__colsample_bytree': [0.8, 0.9, 1.0],
    },
}

## Checking Null and Infinite

In [ ]:
# Cek NaN dan Inf di X_train
print(f"Jumlah NaN di X_train: {pd.isna(X_train).sum().sum()}")
X_train_numeric = X_train.select_dtypes(include = ['number'])
print(f"Jumlah Inf di X_train: {(np.isinf(X_train_numeric).sum().sum())} \n")

# Cek NaN dan Inf di y_train
print(f"Jumlah NaN di y_train: {pd.isna(y_train).sum()}")
y_train_numeric = y_train
print(f"Jumlah Inf di y_train: {(np.isinf(y_train_numeric).sum().sum())}")

In [ ]:
# Cek NaN dan Inf di X_test
print(f"Jumlah NaN di X_test: {pd.isna(X_test).sum().sum()}")
X_test_numeric = X_test.select_dtypes(include = ['number'])
print(f"Jumlah Inf di X_test: {(np.isinf(X_test_numeric).sum().sum())} \n")

## Comparing stage 2

In [ ]:
# 
not_in_x_test = set(X_train.columns) - set(X_test.columns)
not_in_x_train = set(X_test.columns) - set(X_train.columns)

print(f"Kolom yang ada di X_train tapi tidak ada di X_test: {not_in_x_test}")
print(f"Kolom yang ada di X_test tapi tidak ada di X_train: {not_in_x_train}")

In [ ]:
# 
X_train = X_train.drop(columns = not_in_x_test)
X_test = X_test.drop(columns = not_in_x_train)

# 
not_in_x_test = set(X_train.columns) - set(X_test.columns)
not_in_x_train = set(X_test.columns) - set(X_train.columns)

print(f"Kolom yang ada di X_train tapi tidak ada di X_test: {not_in_x_test}")
print(f"Kolom yang ada di X_test tapi tidak ada di X_train: {not_in_x_train}")

## Implement module

In [ ]:
# Hyperparameter Tuning and Model Selection
best_models = {}
for model_name, pipeline in pipelines.items():
    print(f"\nTuning hyperparameters for {model_name}...")

    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv = 5, scoring = 'accuracy', n_jobs = -1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search
    
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best Cross-Validation Score for {model_name}: {grid_search.best_score_}")

## Evaluation and Tuning

In [ ]:
# Model Evaluation on Test Data
for model_name, grid_search in best_models.items():
    print(f"\nEvaluating {model_name} on test data...")

    y_pred = grid_search.best_estimator_.predict(X_test)

    print(f"Classification Report for {model_name}:")
    print(classification_report(y_test, y_pred))